# Cloud Statistics

In [1]:
import geemap
import subprocess
import ee
import os
import pandas as pd
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl
geemap.set_proxy(port=10809)
ee.Initialize()

In [2]:
Map = geemap.Map(center=[34, 99], zoom=4, add_google_map=True)
China_city = ee.FeatureCollection('projects/ee-mrwurenzhe/assets/ChinaShp/China_city_utf8')
vis_params = {
    'color': '000000',
    'colorOpacity': 1,
    'pointSize': 3,
    'pointShape': 'circle',
    'width': 2,
    'lineType': 'solid',
    'fillColorOpacity': 0.66,
}
palette = ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
Map.add_styled_vector(
    China_city, column="FIRST_省�", palette=palette, layer_name="China Counties", **vis_params) # column="地市"
Southest_Tibetan = ee.FeatureCollection(China_city.filter(ee.Filter.inList('地市',ee.List(["林芝市", "昌都市","山南市","拉萨市"]))))
Map.addLayer(Southest_Tibetan, {}, 'Southest_Tibetan')
Map

Map(center=[34, 99], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(chil…

In [16]:
#China_city.reduceColumns(ee.Reducer.toList(),["地市"])
try :
    roi = Map.draw_features[0]
except:
    roi = Southest_Tibetan.union()

## Landsat

In [22]:
L8_T1_TOA = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterBounds(roi.geometry())

# L8_T1_TOA.first().propertyNames()  
CLOUD_COVER = L8_T1_TOA.reduceColumns(ee.Reducer.toList(),["CLOUD_COVER"]).get("list").getInfo()
CLOUD_COVER_LAND = L8_T1_TOA.reduceColumns(ee.Reducer.toList(),["CLOUD_COVER_LAND"]).get("list").getInfo()
DATE_ACQUIRED =  L8_T1_TOA.reduceColumns(ee.Reducer.toList(),["DATE_ACQUIRED"]).get("list").getInfo()
UTM_ZONE = L8_T1_TOA.reduceColumns(ee.Reducer.toList(),["UTM_ZONE"]).get("list").getInfo()
WRS_PATH = L8_T1_TOA.reduceColumns(ee.Reducer.toList(),["WRS_PATH"]).get("list").getInfo()
WRS_ROW = L8_T1_TOA.reduceColumns(ee.Reducer.toList(),["WRS_ROW"]).get("list").getInfo()
Systemindex = L8_T1_TOA.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()
DATE_PRODUCT_GENERATED = L8_T1_TOA.reduceColumns(ee.Reducer.toList(),["DATE_PRODUCT_GENERATED"]).get("list").getInfo()
DATE_PRODUCT_GENERATED = [ee.Date(each).format("YYYY-MM-dd").getInfo() for each in DATE_PRODUCT_GENERATED]

def Create_Pandas(**args):
    return pd.DataFrame(args)
Pd_DataFrame = Create_Pandas(CLOUD_COVER =CLOUD_COVER,
              CLOUD_COVER_LAND=CLOUD_COVER_LAND,DATE_ACQUIRED=DATE_ACQUIRED,
             UTM_ZONE=UTM_ZONE,WRS_PATH=WRS_PATH,WRS_ROW=WRS_ROW,Systemindex=Systemindex,DATE_PRODUCT_GENERATED=DATE_PRODUCT_GENERATED)

# Pd_DataFrame.to_csv(r'C:\Users\A\Desktop\2024铁道学会报奖\L01MSS_C2T1L2_SR.csv')

## Sentinel-SR

In [ ]:
Sentinel2_T1_SR = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterBounds(roi.geometry()).filterDate('2022-01-01', '2023-01-01')

In [ ]:
CLOUDY_PIXEL_PERCENTAGE = Sentinel2_T1_SR.reduceColumns(ee.Reducer.toList(),["CLOUDY_PIXEL_PERCENTAGE"]).get("list").getInfo()
CLOUD_COVERAGE_ASSESSMENT = Sentinel2_T1_SR.reduceColumns(ee.Reducer.toList(),["CLOUD_COVERAGE_ASSESSMENT"]).get("list").getInfo()
CLOUD_SHADOW_PERCENTAGE = Sentinel2_T1_SR.reduceColumns(ee.Reducer.toList(),["CLOUD_SHADOW_PERCENTAGE"]).get("list").getInfo()
MEAN_SOLAR_AZIMUTH_ANGLE = Sentinel2_T1_SR.reduceColumns(ee.Reducer.toList(),["MEAN_SOLAR_AZIMUTH_ANGLE"]).get("list").getInfo()
MEAN_SOLAR_ZENITH_ANGLE = Sentinel2_T1_SR.reduceColumns(ee.Reducer.toList(),["MEAN_SOLAR_ZENITH_ANGLE"]).get("list").getInfo()
# MEDIUM_PROBA_CLOUDS_PERCENTAGE = Sentinel2_T1_SR.reduceColumns(ee.Reducer.toList(),["MEDIUM_PROBA_CLOUDS_PERCENTAGE"]).get("list").getInfo()
# NODATA_PIXEL_PERCENTAGE = Sentinel2_T1_SR.reduceColumns(ee.Reducer.toList(),["NODATA_PIXEL_PERCENTAGE"]).get("list").getInfo()
# NOT_VEGETATED_PERCENTAGE = Sentinel2_T1_SR.reduceColumns(ee.Reducer.toList(),["NOT_VEGETATED_PERCENTAGE"]).get("list").getInfo()
# SNOW_ICE_PERCENTAGE = Sentinel2_T1_SR.reduceColumns(ee.Reducer.toList(),["SNOW_ICE_PERCENTAGE"]).get("list").getInfo()
# THIN_CIRRUS_PERCENTAGE = Sentinel2_T1_SR.reduceColumns(ee.Reducer.toList(),["THIN_CIRRUS_PERCENTAGE"]).get("list").getInfo()
# VEGETATION_PERCENTAGE = Sentinel2_T1_SR.reduceColumns(ee.Reducer.toList(),["VEGETATION_PERCENTAGE"]).get("list").getInfo()
# WATER_PERCENTAGE = Sentinel2_T1_SR.reduceColumns(ee.Reducer.toList(),["WATER_PERCENTAGE"]).get("list").getInfo()
Systemindex = Sentinel2_T1_SR.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()

In [ ]:
def Create_Pandas(**args):
    return pd.DataFrame(args)
Pd_DataFrame = Create_Pandas(CLOUDY_PIXEL_PERCENTAGE=CLOUDY_PIXEL_PERCENTAGE,
                                CLOUD_COVERAGE_ASSESSMENT=CLOUD_COVERAGE_ASSESSMENT,CLOUD_SHADOW_PERCENTAGE=CLOUD_SHADOW_PERCENTAGE,
                                MEAN_SOLAR_AZIMUTH_ANGLE=MEAN_SOLAR_AZIMUTH_ANGLE,
                                MEAN_SOLAR_ZENITH_ANGLE=MEAN_SOLAR_ZENITH_ANGLE,Systemindex=Systemindex)

In [ ]:
Pd_DataFrame.to_csv(r'E:\Desktop\Rs_journal\S2_L2A_SR_22.csv')

## Sentinel-TOA

In [12]:
Sentinel2_T1_TOA = ee.ImageCollection("COPERNICUS/S2_HARMONIZED").filterBounds(roi.geometry()).filterDate('2022-01-01', '2023-01-01')

In [13]:
CLOUDY_PIXEL_PERCENTAGE = Sentinel2_T1_TOA.reduceColumns(ee.Reducer.toList(),["CLOUDY_PIXEL_PERCENTAGE"]).get("list").getInfo()
CLOUD_COVERAGE_ASSESSMENT = Sentinel2_T1_TOA.reduceColumns(ee.Reducer.toList(),["CLOUD_COVERAGE_ASSESSMENT"]).get("list").getInfo()
MEAN_SOLAR_AZIMUTH_ANGLE = Sentinel2_T1_TOA.reduceColumns(ee.Reducer.toList(),["MEAN_SOLAR_AZIMUTH_ANGLE"]).get("list").getInfo()
MEAN_SOLAR_ZENITH_ANGLE = Sentinel2_T1_TOA.reduceColumns(ee.Reducer.toList(),["MEAN_SOLAR_ZENITH_ANGLE"]).get("list").getInfo()
Systemindex = Sentinel2_T1_TOA.reduceColumns(ee.Reducer.toList(),["system:index"]).get("list").getInfo()

In [14]:
def Create_Pandas(**args):
    return pd.DataFrame(args)
Pd_DataFrame = Create_Pandas(CLOUDY_PIXEL_PERCENTAGE=CLOUDY_PIXEL_PERCENTAGE,
              CLOUD_COVERAGE_ASSESSMENT=CLOUD_COVERAGE_ASSESSMENT,MEAN_SOLAR_AZIMUTH_ANGLE=MEAN_SOLAR_AZIMUTH_ANGLE,
            MEAN_SOLAR_ZENITH_ANGLE=MEAN_SOLAR_ZENITH_ANGLE,Systemindex=Systemindex)

In [15]:
Pd_DataFrame.to_csv(r'E:\Desktop\Rs_journal\S2_L1C_TOA_22.csv')

# CSV合并
按顺序合并一个文件夹中的所有CSV文件

In [42]:
Dir = r'E:\Desktop\Rs_journal\S2_L2A_SR'
CSV_Paths = sorted([os.path.join(Dir,f) for f in os.listdir(Dir) if f.endswith('.csv')],key=lambda info: int(info.split('_')[-1].split('.')[0]))
for i,each in enumerate(CSV_Paths):
    if i == 0:
        DataFrame = pd.read_csv(each)
    else:
        DataFrame = pd.concat([DataFrame, pd.read_csv(each)], axis=0, ignore_index=True)
DataFrame.to_csv(os.path.join(Dir,os.path.basename(Dir)+'_ALL.csv'))
print('=={}== is saved'.format(os.path.join(Dir,os.path.basename(Dir)+'_ALL.csv')))